Imports and fucntions to calc ALS

In [ ]:
from pyspark.sql import functions as f, types as t, Window
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.recommendation import ALSModel
from pyspark.ml.feature import StringIndexer


def load(paths, has_rating=True):
    if not isinstance(paths, list):
        paths = [paths]
    result = (
        spark.read.csv(paths, header=True)
        .withColumn('userId', f.col('userId').cast('int'))
        .withColumn('movieId', f.col('movieId').cast('int'))
        .withColumn('timestamp', f.col('timestamp').cast('int'))
        .filter(f.col('userId').isNotNull())
    )
    if has_rating:
        result = result.withColumn('rating', f.col('rating').cast('float'))
    return result.cache()

Load model and data

In [ ]:
user_movie_ALS = ALSModel.load('/user/mob2019014/user_movie_ALS_model.bin')

ratings_test = load('/data/MobodMovieLens/test/ratings.csv', has_rating=False)

Make predictions

In [ ]:
ratings_with_ALS = user_movie_ALS.transform(ratings_test)

Load them on disk from hdfs

In [ ]:
import subprocess

command = (
    "hdfs dfs -cat "
    "$(hdfs dfs -ls ratings_test_with_all_ALS_predictions.csv | awk '{if (NR == 3) print $8;}')"
    " > ratings_test_with_all_ALS_predictions.csv"
)
subprocess.call(command, shell=True)

Code to load data from google drive

In [3]:
import pandas as pd
als_final_test = pd.read_csv('ratings_test_with_all_ALS_predictions.csv')

In [5]:
als_final_test['prediction'] = als_final_test['user_movie_ALS']
als_final_test.fillna(3.5).to_csv(
    'prediction.csv',
    index=False,
    columns=['movieId', 'userId', 'timestamp', 'prediction']
)